#TEMA: Previsão dos preços das ações usando Python 

#O tema que vou abordar é uma previsão dos preços das ações da empresa “The Walt Disney Company'', com objetivo de entender o melhor momento para investir nas ações. A base de dados utilizada foi extraída do “Yahoo Finance’ . Em resumo, os dados encontrados nesse Data Freme são o Date, referente a data que estou analisando, o Open é a abertura das ações, o High é o momento mais alto que as ações chegaram, enquanto o Low o momento mais baixo. Agora falando o Close é o fechamento que as ações tiveram, o Adj Close é Ajuste diário é uma equalização automática da posição das pessoas que investem no Mercado Futuro, e por fim o Volume é referente ao Volume são basicamente o número de ativos que foram comprados e vendidos em determinado dia. 

#Esse projeto tem como objetivo prever os preços das ações da empresa The Walt Disney Company, dona das franquias Star Wars, Marvel, Mickey Mouse, entre outros. 

#Nessa etapa vou importar as bibliotecas do Python necessárias, e coleta os dados mais recentes das ações da empresa Disney:

In [30]:
#Conexão com o dataset e importando bibliotecas:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('DIS', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", 
             "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)


#A conexão com o dataset foi realizada através biblioteca yfinance, que faz conexão direta no Yahoo Finance.

In [31]:
#Exibição do trecho dos dados:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3443,2022-05-23,103.760002,106.029999,101.559998,105.830002,105.830002,14402400
3444,2022-05-24,103.709999,103.839996,100.129997,101.589996,101.589996,13970200
3445,2022-05-25,101.430000,103.809998,101.160004,103.260002,103.260002,11026600
3446,2022-05-26,104.220001,106.129997,103.959999,105.610001,105.610001,11297800
3447,2022-05-27,106.889999,109.370003,106.529999,109.320000,109.320000,13660700




#Com gráfico de Candlestick pode-se visualizar com mais clareza o aumento e diminuição dos preços das ações:

In [36]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Disney Stock Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

#Correlação de todas as colunas com a coluna Fechar, pois é a coluna de destino:

In [38]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
High         0.999809
Low          0.999783
Open         0.999618
Adj Close    0.999092
Volume      -0.127191
Name: Close, dtype: float64


#Foi necessário treinar um modelo LSTM, com objetivo de prever os preços das ações, primeiramente dividir os dados em treino e teste:

In [39]:
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)


#Com isso vou preparar uma arquitetura de rede neural para LSTM:

In [40]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 4, 128)            66560     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 25)                1625      
                                                                 
 dense_5 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________



#Nessa etapa podemos treinar nosso modelo de rede neural para prever os preços das ações:

In [18]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=30)

Epoch 1/30
2758/2758 [==============================] - 30s 4ms/step - loss: 196.6047
Epoch 2/30
2758/2758 [==============================] - 11s 4ms/step - loss: 17.8701
Epoch 3/30
2758/2758 [==============================] - 11s 4ms/step - loss: 11.4931
Epoch 4/30
2758/2758 [==============================] - 11s 4ms/step - loss: 9.4680
Epoch 5/30
2758/2758 [==============================] - 11s 4ms/step - loss: 11.4744
Epoch 6/30
2758/2758 [==============================] - 11s 4ms/step - loss: 9.5451
Epoch 7/30
2758/2758 [==============================] - 11s 4ms/step - loss: 7.4562
Epoch 8/30
2758/2758 [==============================] - 11s 4ms/step - loss: 6.3614
Epoch 9/30
2758/2758 [==============================] - 11s 4ms/step - loss: 6.5213
Epoch 10/30
2758/2758 [==============================] - 11s 4ms/step - loss: 7.1940
Epoch 11/30
2758/2758 [==============================] - 11s 4ms/step - loss: 6.3383
Epoch 12/30
2758/2758 [==============================] - 12s 4ms/step

#Com isso vamos testar o modelo, fornecendo valores de entrada de acordo com os recursos, que usei para treinar o modelo e prever o resultado final:

In [11]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[177.089996, 180.419998, 177.070007, 74919600]])
model.predict(features)

array([[181.86398]], dtype=float32)

#Desse modo pode-se usar a arquitetura de rede neural LSTM para a tarefa de previsão de preços de ações da empresa Disney.